In [3]:
import pickle
import numpy as np
import pandas as pd

In [4]:
with open('data/question_data_emb', 'rb') as f:
    all_sets, embs, word2idx = pickle.load(f)

In [6]:
train = all_sets[0]

In [7]:
q1 = train['q1']

In [26]:
train.keys()

dict_keys(['q1', 'q2', 'y'])

In [18]:
size = [len(x) for x in q1]

In [21]:
temp = pd.DataFrame(size)

In [25]:
temp.describe()

,0
count,323429.000000
mean,12.540839
std,6.152808
min,1.000000
25%,9.000000
50%,11.000000
75%,15.000000
max,146.000000


In [28]:
embs.shape

(68547, 300)

In [9]:
type(q1)

list

In [10]:
len(q1)

323429

In [11]:
question_data = pd.read_csv('data/train_dqd.csv')

In [12]:
question_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [13]:
question_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404287 entries, 0 to 404286
Data columns (total 6 columns):
id              404287 non-null int64
qid1            404287 non-null int64
qid2            404287 non-null int64
question1       404287 non-null object
question2       404287 non-null object
is_duplicate    404287 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB
